In [392]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import chart_studio.plotly as py

In [393]:
# Reading csv files
df = pd.read_csv('data/coronavirus_cases_usa.csv', index_col=0)
df_coronavirus = pd.read_csv('data/coronavirus_searches_usa.csv', index_col=0)
df_worldometers = pd.read_csv('data/worldometers.csv', index_col=0)

# Merging Google Trends popularity dataframes
df_searches = pd.merge(df_coronavirus, df_worldometers)

# Conversion to datatime and set as index
df_searches['Week'] = pd.to_datetime(df_searches.Week)
df['Date'] = pd.to_datetime(df.Date)
df_searches = df_searches.set_index('Week')
df = df.set_index('Date')

# Resampling and interpolation of trends
df_searches = df_searches.asfreq('D').interpolate(method='polynomial', order=2)

# Merging cases and Google Trends
df = df.merge(df_searches, left_index=True, right_index=True)

In [404]:
# Visualization
fig = make_subplots(specs=[[{"secondary_y": True}]])

# USA COVID19 Cases
fig.add_trace(
    go.Scatter(
        x=df.index, 
        y=df.Cases.rolling(7).mean(), 
        name="USA Cases",
        marker=dict(color="#0D3D56"),
        showlegend=True,
        hovertemplate="Daily: %{y:.0f}<br>"),
    secondary_y=False,
)

# Coronavirus Google Trends
fig.add_trace(
    go.Scatter(
        x=df.index, 
        y=df.coronavirus_searches, 
        mode='lines',
        name='"Coronavirus"',
        marker=dict(color="#007bff"),
        showlegend=True,
        hovertemplate="Popularity: %{y:.0f}<br>"
    ),
    secondary_y=True,
)

# Worldometer Google Trends
fig.add_trace(
    go.Scatter(
        x=df.index, 
        y=df.Worldometer, 
        name='"Worldometers"',
        marker=dict(color='#F16E00'),
        showlegend=True,
        text='"Coronavirus" searches',
        hovertemplate="Popularity: %{y:.0f}<br>"
    ),
    secondary_y=True,
)

# Adding buttons to select time period
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7,
                     label="7d",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=3,
                     label="3m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=False
        ),
        type="date"
    )
)

# Adding axes titles and legend
fig.update_layout(
    margin={'l': 0, 'r': 0, 't': 0, 'b': 0},
    yaxis=dict(title_text="Cases", titlefont=dict(size=16), showgrid=False, fixedrange=True),
    legend=dict(orientation='h', y=-0.1),
    hovermode="x",
    dragmode="pan"
    
)
fig.update_yaxes(title_text="Search Popularity", titlefont=dict(size=16), showgrid=False, fixedrange=True, secondary_y=True)

# Exporting to Chart Studio
py.plot(fig, filename = 'usa-covid-google-trends', auto_open=True)

'https://plotly.com/~hugolmn/55/'